# Lab 2B: Functional programming, and declarative patterns


---
**GroupNo: 4**


**Student:** mimte666

**Student:** steto820

---

Disclaimer: Functional programming in Python does not always lead to the fastest possible code, and is often not considered the *pythonic* approach. However, functional programming is the basis for many concurrent systems (the MapReduce programming model which many big data systems, e.g. Hadoop, relies on gets its name from the *map* and *reduce* functions mentioned below). Python is a multi-paradigm language, and functional programming is one of the main paradigms one can use. To understand how and when to do this, it is necessary to do things in a non-*pythonic* way in order to cover the basics.

## General instructions

In this lab there are some general rules you should keep in mind to make sure you are on the correct path in your solutions.

#### Rules
1. You are not allowed to use `while` or `for` statements unless this is explicitly allowed in the task.
2. You are not allowed to use global variables (other than for functions defined in the global environment).
3. Code stubs should be viewed as fixed, you are only allowed to add code, the only code you are allowed to change is `pass` statements, which you should remove.
4. You should refrain from using the `list` datatype unless otherwise specified and instead use `tuple`. One of the strengths of functional programming is its focus on immutable data types (this is why functional programming and concurrency goes so well together). Incidentally, one might find speedups when using the immutable tuples instead of lists.

#### Advice
1. Avoid local variables unless you are certain they are necessary, in most cases you won't need to use local variables. (altermatively, use local variables to your hearts content, but when your solution works, try to eliminate them, you should be able to eliminate most of them, over time, you might find that you don't need them.)

# 2 Recursion

As an introduction to linear recursion, read the introductory note on the course webpage. This might help explain terms that you may not know (even if the concept is previously known).

## 2.1 Linear recursion

a) Write a recursive function `sum_even(n)` that takes a natural number $n\geq 0$ and returns the sum of all even numbers $0,...,n$. It should be linear-recursive with delayed computations.

In [ ]:
def sum_even(n):
    if(n==0):
        return(n)
    if(n%2==1):
        return(n - 1 + sum_even(n-3))
    else:
        return(n + sum_even(n-2))
sum_even(6)

b) Write `sum_even_it(n)` according to the same specification. In this case, the solution should be tail recursive.

In [ ]:
# Your code here.
def sum_even_it(n, a=0):
    if(n==0):
        return(a)
    if(n%2==1):
        return(sum_even_it(n-3, a + n - 1))
    else:
        return(sum_even_it(n-2, a + n))

sum_even_it(7)

c) We can of course express this in a declarative and Pythonic way, which is non-recursive. Write a function `sum_even_py` which returns the same result as above, but using comprehension or filter/map/reduce construct.

In [ ]:
def sum_even_py(n):
    return(sum(filter(lambda x: x%2==0, range(n+1))))
sum_even_py(7)

## 2.2 Double/tree recursion

Sometimes we might find ourselves with branching structures, where there are several "smaller" cases to recurse over. This might for instance be the case when we have trees, lists-within-lists or the like.

[Note: In the tasks below, it might be helpful to gain an understanding of `isinstance`. See the documentation!]

a) One common use of recursion is to traverse recursive data structures. One exercise might be to _flatten_ nested lists or tuples. This is relatively simple with only one level of nesting, or when the structure follows a strict pattern, but for arbitrary nested sequences, a recursive approach is more natural. Implement a recursive function `myflatten` which can take an arbitrary structure of nested tuples and flattens it (in the sense of returning a new non-nested tuple with the same elements in the same order).

In [ ]:
def myflatten(seq):
    if not seq:
        return seq
    elif isinstance(seq[0], tuple):
        return myflatten(seq[0]) + myflatten(seq[1:])
    return tuple((seq[0],)) + myflatten(seq[1:])
    
def test_myflatten():
    tests = (
     ((), (), "the empty tuple"), 
     ((1,2,3), (1,2,3), "flat tuples invariant under flattening"), 
     ( (1, (2), 3, (4, 5, (6), 7), 8), (1, 2, 3, 4, 5, 6, 7, 8), "Arbitrarily nested tuples.")
    )
    for arg, expected_output, error_msg in tests:
        assert myflatten(arg) == expected_output, error_msg
    print("test_myflatten run successfully!")

test_myflatten()    # Uncomment this line to run tests.

b) Implement a function `exists_in(e, seq)` which returns `True` if the element `e` exists somewhere in the tuple `seq` (and `False` otherwise). `seq` might be nested and contain tuples-within-tuples.

In [ ]:
def exists_in(e, seq):
    if not seq:
        return False    
    elif isinstance(seq[0], tuple):
        return True if exists_in(e, seq[0]) else exists_in(e, seq[1:])
    else:
        return True if seq[0] == e else exists_in(e, seq[1:])
    
exists_in(1, (2,3,(2,3,(21)),1))


c) Write a few representative test cases, as in lab 2A.

In [ ]:
def test_find_anywhere():
    tests = (
        ((32442,()),False,'empty tuple'),
        ((1,(2,3,(6,(1,4)),5)),True,'expected item could be found in nested tuple'),
        ((1,(2,3,4)),False,'expected item could not be found in nested tuple'),
    )
    
    for arg, expected_output, error_msg in tests:
        assert exists_in(*arg) == expected_output, error_msg
    print("test find anywhere function run! successfully")

test_find_anywhere()

d) One of the most famous recursive functions is the Quicksort function (https://en.wikipedia.org/wiki/Quicksort). It allows us to sort a sequence, with repeated values, in (amortized) log-linear time and with a logarithmic number of recursive calls. We will start by implementing Quicksort for a tuple of numbers.

Note that Wikipedia illustrates a more advanced _in-place_ version of Quicksort, with a more advanced partition function. For the purposes of this assignment, you can simply pass a new tuple or generator to each recursive call to quicksort. You may use eg _filter_ or a comprehension to create the inputs.

In [74]:
from random import sample, choice

# with comprehension
def quicksort(part):
    if not part:
        return part
    pivot = part[0]
    left = [x for x in part[1:] if x<pivot]
    right = [x for x in part[1:] if x>=pivot]
    return(quicksort(left) + list([pivot]) + quicksort(right))



a = tuple(sample(range(1000,2000), 1000))
#print(a)
b = quicksort(a)
print(b==sorted(a))


True


## 3 Higher order functions (HOF)
 
A _higher-order function_ is a function which operates on other functions. What this means exactly is disputed, but we will call any function which returns a function or takes a function as an argument a higher-order function. (Conversely, a function neither taking another function as input nor returning a function we will refer to as a _first-order function_)

In R you have encountered these when, for instance, using the `apply` family of functions, which are all versions of what is called a `map` function in functional programming (see below).

When using higher-order functions, it is often useful to create simple anonymous functions at the place in the code where they are used, rather than defining a new named function in one place only to call it in a single other place. In R, all functions are created in this way with the `function` keyword, but they are usually assigned to global names with standard assignment (`<-`). Python provides similar functionality using the `lambda` keyword (name inspired by Alonzo Church's [$\lambda$-calculus](https://www.youtube.com/watch?v=eis11j_iGMs) which has inspired much of functional programming) with which we can create anonymous functions. Of course, we can also pass named functions to higher-order functions, which is usually the case when the function is predefined, general enough to be used in more than one place, or complex enough to warrant separate definition and documentation for the sake of clarity.

## 3.1 The three standard functions `map`, `reduce` and `filter`

There are three standard cases which are widely applicable and many other higher-order functions are special cases or combinations of these. They are: `map`, apply a function on each element in a sequence, `filter`, keep (or conversely, remove) elements from a sequence according to some condition, and `reduce`, combine the elements in a sequence. The `map` function takes a sequence and a function (usually of 1 parameter) which is to be applied to each element of the sequence and might return anything, this function is assumed not to have side effects. The `filter` function takes a function (usually of 1 parameter) which returns a boolean value used to indicate which elements are to be kept. The `reduce` function takes a function (usually of 2 parameters) which is used to combine the elements in the sequence.

In Python, `map` and `filter` are standard built-in functions. Since Python 3, the `reduce` function needs to be imported from the `functools` module.

Many more advanced functions, of any order, can be created by combining these three higher-order functions.

A note from last year: usually, the `reduce` function is more difficult to grasp than `map` and `filter` but I found this blog-post by André Burgaud to be a nice introduction to `reduce`. Note that Burgaud talks about the more general _fold_ concept rather than `reduce`, which is a special case of fold often called _left fold_ (this is covered in more detail in the post). https://www.burgaud.com/foldl-foldr-python/

a) Implement a function `mysum` which computes the sum of a list or tuple of numbers using the reduce function and a lambda function.

In [2]:
from functools import reduce

def mysum(seq):
    return reduce(lambda x,y: x+y, seq)

mysum((4, 7, 1))

12

b) Implement a function `mylength` which uses `map` and `reduce` to compute the length of a sequence. The use of the `len` function is not allowed.

[Hint: Use `map` to convert the input to something which can easily be `reduce`:d.]

In [3]:
def mylength(seq):
    return reduce(lambda x,y: x+y, map(lambda x:1, seq))
    
print(mylength((4, 2, 5, 2, 5)))
print(mylength("test"))

5
4


## 3.2 Building your own higher order functions

a) Re-implement the three basic functional helper functions `map`, `filter` and `reduce` **as purely functional recursive functions**. You may not express this as eg comprehensions; the task is to practice figuring out this type of logic.

Note that the built-in versions of these functions work on multiple sequences of equal length if supplied, however, you can assume a single sequence as second parameter, i.e. you can also skip the third parameter to reduce.

In [ ]:
def mymap(f, seq):
    if not seq:
        return seq
    else:
        return tuple((f(seq[0]),)) + mymap(f, seq[1:])

mymap(lambda x:x**2, tuple(range(10)))

In [ ]:
def myfilter(f, seq):
    if not seq:
        return seq
    else:
        return tuple((seq[0],) if f(seq[0]) else ()) + myfilter(f, seq[1:])

myfilter(lambda x:x%2==0, tuple(range(10)))

You might note the similarities with how you implemented `sum_even`.

In [ ]:
def myreduce(f, seq):
    if len(seq)==2:
        return f(seq[0], seq[1])
    else:
        return myreduce(f, tuple((f(seq[0], seq[1]),)) + seq[2:])

myreduce(lambda x, y: x*y, tuple(range(1,7)))

## 3.3 Returning functions

The previous section covered functions which take other functions as input, but what about the opposite, functions returning functions as output?

a) Function composition is a common in both maths and programming. Write a function `compose` which takes two functions, $f$ and $g$, and produces the _composite_ function $f \circ g$, where $(f \circ g)(x) \Leftrightarrow f(g(x))$. Example use is given below.

In [ ]:
from statistics import stdev, mean

def compose(f, g):
    return lambda x: f(g(x))

def myscale(vals):
    return [x/stdev(vals) for x in vals]

def myshift(vals):
    return [x-mean(vals) for x in vals]

standardize = compose(myscale, myshift)

print(standardize(range(-3, 8)))

b) Create a function `composition(*funs)` which takes a non-empty sequence of functions of one argument and returns their sequential composition. That is $composition(f_0,f_1, \ldots, f_n) = f_0 \circ f_1 \circ\ldots \circ f_n$. (The question of if $f\circ g \circ h$ should be read $f\circ (g\circ h)$ or $(f \circ g) \circ h$ is perfectly valid, but they turn out to be the same. That is, $\circ$ is associative.)

In [ ]:
def composition(*funs):
    return reduce(compose, funs)

standardize = composition(myscale, myshift)
print(standardize(range(-3, 8)))

Hint: Don't remember what can be found in `*funs`? Print it! Don't know how the values should be combined? Write out some simple example on paper.

Note: This task demonstrates the generality of our constructs. Previously we worked with sequences of numbers and the like. Now we lift this to the level of working with functions as values, and instead of using combinators which work on numbers, we use function combinators in conjunction with our known patterns.

### Voluntary task: pipelining

When doing data analysis, one very important part is pre-processing. Often, data goes through a number of steps of preprocessing, sometimes called a pipeline. The function composition example above can be seen as a special case of such a pipeline for only two functions. By clever use of higher order functions, we can build a pipeline function which takes a list or tuple of data transforming functions and creates a function which applies these sequentially. Construct such a function called `make_pipeline`. In order to focus on the primary purpose of the `make_pipeline` function, we will perform a very simple set of transformations, increment each value by 1, take the absolute value, and then take the square root. Usage example and code for the `inc` function is supplied below.

You may want to use functions you have defined above.

In [ ]:
from functools import reduce, partial
from math import sqrt 


def make_pipeline(*funs):
    return lambda vals: ???

# We can even drop the lambda vals : bit, using partial
# evaluation (see the help for functools.partial!)

def inc(x):
    return x+1

pipeline = make_pipeline(inc, abs, sqrt)

tuple(pipeline(range(-5,5)))

## 4. Simple declarative Pythonic patterns (involing HOF)

a) As preparation, create a named tuple type "coord" which has fields `x` and `y`.

In [3]:
from collections import namedtuple
coord = namedtuple("coord", [
    "x",
    "y"
])

five_three = coord(5,3)
assert five_three.x == 5, "first element is the x coordinate"
assert five_three.y == 3, "the second element is the y coordinate"

b) Generate a $10^7$ random coordinates, with $x$ and $y$ coordinates drawn uniformly from [-1000,1000]. Save the tuple of those with $x + y > 0$ as `rnd_coords`. How many are there?

In [4]:
from random import uniform

coord_sample = tuple(map(lambda x: coord(uniform(-1000,1000),uniform(-1000,1000)), range(10**3)))
len(coord_sample)

1000

In [5]:
rnd_coords = tuple(filter(lambda coord: coord.x + coord.y > 0, coord_sample))
len(rnd_coords)

519

[Note: If this takes a while, you might want to consider when the elements are generated and saved.]

**Before having solved the tasks below, consider setting `coords` to a smaller set (eg generate $10^3$ elements instead of $10^7$ to start with).**

c) Let `sorted_rnd` be the coordinated sorted first by the `x` component and then the `y`. Use a built-in Python sorting function. Do you need any extra parameters? Why? Why not? How would you find out where the order comes from (and might it be consistent but useless, eg sorting the elements by memory location)?

In [16]:
a = [coord(1,9),coord(1,1), coord(1,5), coord(2,3),coord(0,1)]

sorted(a)

[coord(x=0, y=1),
 coord(x=1, y=1),
 coord(x=1, y=5),
 coord(x=1, y=9),
 coord(x=2, y=3)]

In [6]:
#sorted_rnd = sorted(rnd_coords)
#sorted_rnd
## We can see we do not need any parameter to sort this data. It sorted it directly
## by the x with a increasing order and if we have some same x values, it will sort 
## hem by y with increasing order. 
## If we want to change this we should use the line below:
## sorted_rnd = sorted(rnd_coords, key=lambda p:-p.x, p.y)

[coord(x=-882.8832259427566, y=959.8436976366106),
 coord(x=-851.7264608198407, y=876.1981663514696),
 coord(x=-817.828022227185, y=822.0129524121824),
 coord(x=-812.7425080236123, y=837.9913153638465),
 coord(x=-794.0538671270411, y=807.8877568489802),
 coord(x=-777.50237781489, y=912.224104307076),
 coord(x=-772.008074032793, y=825.3452519640457),
 coord(x=-760.5861947983203, y=989.7652218707981),
 coord(x=-760.0305730426389, y=857.5070316175791),
 coord(x=-745.3881691077765, y=998.5809639220795),
 coord(x=-741.8922252037394, y=859.3113328257477),
 coord(x=-741.1163655800001, y=940.2824413351716),
 coord(x=-716.1151369957092, y=902.0438652657592),
 coord(x=-687.1742609125808, y=780.6169963213201),
 coord(x=-660.2242253974932, y=714.7084743857856),
 coord(x=-659.5066654743617, y=955.2887466446025),
 coord(x=-655.1487039891342, y=730.8442072787786),
 coord(x=-634.4664650647949, y=903.8896852464229),
 coord(x=-627.4238066859225, y=800.9210899443203),
 coord(x=-625.6578357391788, y=796.1

[General words of advice:

* During testing, you might want to use a smaller data set (and then try it out at a larger set).
* You might not want to display the entire list to see if you're right all the time. Slicing out the first and last elements, say the first or last 10, might provide some hints.
* You could naturally define a function which checks that the list is in order (or performs some probabilistic sampling test), to test this.]

d) Sort the values (in the sense of returning a new sorted tuple) by their Euclidean distance to the point (5,3). Continue using a built-in Python sorting function.

In [ ]:
pts_near_53 = sorted(rnd_coords, key=lambda p: ((p.x-5)**2+(p.y-3)**2)**0.5)
pts_near_53

Note: here we customise the behaviour of a built-in function by passing it information about our intended ordering.

d) Define the function `sorted_by_distance(origo)` which takes a coordinate `origo` and returns a function which sorts the sequence by the euclidean distance to `origo`. (Ie those closest to origo come first in the list.)

In [ ]:
def sorted_by_distance(origo):
    def f(points):
        return(sorted(rnd_coords, key=lambda p: ((p.x-origo.x)**2+(p.y-origo.y)**2)**0.5))
    return f

ordered_by_closeness_to_53 = sorted_by_distance(coord(5,3))   # Return the function.
pts_near_53_2 = ordered_by_closeness_to_53(rnd_coords)     # Applying the function 

assert pts_near_53 == pts_near_53_2

[Note: Here we extend the work above to a higher-order function, which uses the local value of `origo`. In essence, this task summarises higher order functionality - we create a closure, return a function and use a custom ordering ]

e) So far in the course, we have seen, and possibly used `enumerate`, `range`, `zip`, `map` and `filter` as declarative constructs (along with the general comprehension syntax). Now we introduce a further useful iterator construct. Construct something called `reverse_squared` which when prompted would give us the squares of elements 0,...,N _but in reverse_ (that is $N^2, (N-1)^2, ..., 2^2, 1^2, 0^2$).

In [ ]:
# The time it takes to run this shouldn't really depend on if you use SMALL_N or BIG_N.

BIG_N = 99999999
SMALL_N = 999

N = SMALL_N    # change this to test later on

reverse_squares = tuple(map(lambda x:x*x, list(range(N))[::-1]))
reverse_squares

In [ ]:
# Experimentation: copy and paste your code from above into this cell.
# This is rather crude, but we want you to to be able to trust that any
# slowness in the cell above can be found by reference to that code, not the
# profiling code below.

# Copy-pasting as it might be useful to have fresh maps.


import profile

# We cut and paste this code 
BIG_N = 99999999
SMALL_N = 999

N = SMALL_N    
# Look at the run time. Switching from BIG_N to SMALL_N shouldn't really matter.
# This suggests that we have quick access to elements at the end of our (squared) range.

reverse_squares = None # <<----------- Your code from the cell above goes here.


profile.run("print('Did we find it? ', {} in reverse_squares)".format( N**2 ))

Note: once you know of the construct, this task is extremely simple. It mostly serves as a demonstration of the availability of these constructs, and how they can be combined. Also, it points to efficiency considerations when using declarative iterator constructs as opposed to fixed computed structures.

As the profiling code above suggests, where we redefine the object in every run, we do not have a purely functional construct. In that case, we wouldn't be able to exhaust the values.

[Additional reading: some additional tools are available in the `itertools` module.]

## 5 Mutating function state

A function always has access to the environment in which it was created. Usually, this means that the function can access global variables. It also means that it can access and modify local bindings from where it was created.

A closure is a function which has access to an environment which is not accessible from outside the function (but which is not destroyed when the function returns). I.e. it is a way to introduce a small measure of statefulness into functional programming. In Python, iterators and generators work much like this. However, we can use the general concept in many cases.

a) Implement a function `make_counter` which has a single parameter `n` which acts as the initial value for a counter. The function should return a function with no parameters which, when called, increments the value of `n` by 1 and returns the new value.

In [7]:
def make_counter(n):
    def increment():
        nonlocal n
        n += 1
        return(n)
    return increment

counter_A = make_counter(0)
counter_B = make_counter(15)
print("To show that the functions do not affect each others' states, consider the printout:")
print("counter_A returns: {}".format(counter_A()))
print("counter_A returns: {}".format(counter_A()))
print("counter_B returns: {}".format(counter_B()))
print("counter_A returns: {} (was it affected by the call to counter_B?)".format(counter_A()))

To show that the functions do not affect each others' states, consider the printout:
counter_A returns: 1
counter_A returns: 2
counter_B returns: 16
counter_A returns: 3 (was it affected by the call to counter_B?)


## 6. Use case: parametrisation

Above, we see how `sorted` can be parametrised with information about the intended order. We want to extend our `quicksort` to work the same way. We should be able to provide the way for it to tell if object A in the tuple should come before object B, or after. This is done by mapping the objects onto something where we do have an order.

a) Copy your code from the `quicksort` task above, and extend it. Call the function `quicksort_param` for parametrised, and allow a key parameter to be passed in (like in `sorted`). Note that the key function should be optional. We thus want default arguments.

In [55]:
from random import random

# Write quicksort_param here:
def quicksort_param(part, fun=None):
    if not part:
        return part
    
    # add different functionalities to sort function
    myfun = lambda x:x
    if(fun!=None):
        myfun = lambda x:fun(x)

    pivot = part[0]
    left = [x for x in part[1:] if myfun(x)<myfun(pivot)]
    right = [x for x in part[1:] if myfun(x)>=myfun(pivot)]
    return(quicksort_param(left, fun) + [pivot] + quicksort_param(right, fun))


a = tuple(tuple(random() for i in range(3)) for j in range(10))
#print(a)
b = quicksort_param(a, sum)   # Elements are three-tuples. Those with smallest sums of values should come first.
print(b)
print(quicksort_param([5,2,9,200]))   # No key function provided.

[(0.30432974396793855, 0.0541047245967371, 0.6701947492431644), (0.647083701525629, 0.08990857327841417, 0.406886053716197), (0.4626361265691079, 0.6341755289738568, 0.12044502647374189), (0.2356740488604877, 0.9987040747902876, 0.07420645834658157), (0.5029778022754718, 0.25703305289960643, 0.6625508786931115), (0.40912816493299287, 0.35137097479513946, 0.8270069161628679), (0.6069717191556525, 0.7547002793560629, 0.2998501123155709), (0.4324036244587285, 0.6897443875211782, 0.8344422512736559), (0.999520013673843, 0.5565490071453735, 0.7646201608719135), (0.8874888669452529, 0.5505412948607102, 0.9205909112158601)]
[2, 5, 9, 200]


## Attribution

Lab by Johan Falkenjack (2018), extended and rewritten by Anders Märak Leffler (2019).

License [CC-BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)